In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
point = np.zeros((2, 50*102), dtype=np.float32)
h_prev = np.zeros((2, 512), dtype=np.float32)
delta_t = np.ones((2, 50), dtype=np.float32)

In [3]:
class LessThanConstraint(tf.keras.constraints.Constraint):
    def __init__(self, max_value):
        self.max_value = max_value

    def __call__(self, w):
        return tf.clip_by_value(w, clip_value_min=-float('inf'), clip_value_max=self.max_value)

In [4]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, d):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.d = d

    def build(self, input_shape):
        self.W_aq = self.add_weight(name='Waq', shape=(self.hidden_dim * 2, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_ah = self.add_weight(name='Wah', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.ba = self.add_weight(name='ba', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.v = self.add_weight(name='v', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        #self._ati_dense = self.add_weight(name='ati', shape=(input_shape[-2], self.hidden_dim), initializer="random_uniform", trainable=True)
        return super().build(input_shape)

    def call(self, ht, ct, hi):
        q = tf.concat((ht, ct), axis = -1)
        aq = tf.matmul(q, self.W_aq)
        ah = tf.matmul(hi, self.W_ah)

        oti = tf.math.tanh(aq[:, tf.newaxis, ...] + ah + self.ba)

        a = tf.matmul(oti, self.v)
        ati = tf.exp(a)
        ati = ati / tf.reduce_sum(ati, axis=1)[:, tf.newaxis, ...]

        #ati = tf.matmul(ati, self._ati_dense)
        e = tf.reduce_sum(ati * hi, axis=1)
        return e

In [5]:
class InteractionModule(tf.keras.layers.Layer):
    def __init__(self, hidden_dim):
        self.hidden_dim = hidden_dim
        super(InteractionModule, self).__init__()

    def build(self, input_shape):
        self.wh = self.add_weight(name='wh', shape = (self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.we = self.add_weight(name='we', shape = (self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.wg = self.add_weight(name='wg', shape = (input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bh = self.add_weight(name='bh', shape = (1, self.hidden_dim), initializer="random_uniform", trainable=True)
        return super().build(input_shape)

    def call(self, g, ht, e):
        _h = tf.matmul(ht, self.wh)
        _e = tf.matmul(e, self.we)
        _g = tf.matmul(g, self.wg)
        new_h = tf.math.tanh(_h + _e + _g + self.bh)
        return new_h

In [6]:
class cell(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, d, **kwargs):
        super().__init__( **kwargs)
        self.hidden_dim = hidden_dim
        self.attn = Attention(hidden_dim, d)
        self.IM = InteractionModule(hidden_dim)
        self.d = d

    def build(self, input_shape):
        self.W_sh = self.add_weight(name='Wsh', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_sx = self.add_weight(name='Wsx', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_st = self.add_weight(name='Wst', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bs = self.add_weight(name='bs', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_fh = self.add_weight(name='Wfh', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_fx = self.add_weight(name='Wfx', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_fs = self.add_weight(name='Wfs', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True, constraint=LessThanConstraint(max_value=0.0))
        self.bf = self.add_weight(name='bf', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_ih = self.add_weight(name='Wih', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_ix = self.add_weight(name='Wix', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_is = self.add_weight(name='Wis', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bi = self.add_weight(name='bi', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_Th = self.add_weight(name='WTh', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_Tx = self.add_weight(name='WTx', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_Ts = self.add_weight(name='WTs', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bT = self.add_weight(name='bT', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_Eh = self.add_weight(name='WEh', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_Ex = self.add_weight(name='WEx', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_Es = self.add_weight(name='WEs', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bE = self.add_weight(name='bE', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_oh = self.add_weight(name='Woh', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_ox = self.add_weight(name='Wox', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_os = self.add_weight(name='Wos', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bo = self.add_weight(name='bo', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)
        return super().build(input_shape)
    
    def call(self, input, h_prev, c_prev, delta_t, mem):
        sh = tf.matmul(h_prev, self.W_sh)
        sx = tf.matmul(input, self.W_sx)
        st = tf.matmul(delta_t, self.W_st)

        s = tf.tanh(sh + sx + st + self.bs)

        fh = tf.matmul(h_prev, self.W_fh)
        fx = tf.matmul(input, self.W_fx)
        fs = tf.matmul(s, self.W_fs)

        f = tf.sigmoid(fh + fx  + fs + self.bf)

        ih = tf.matmul(h_prev, self.W_ih)
        ix = tf.matmul(input, self.W_ix)
        i_s = tf.matmul(s, self.W_is)

        i = tf.sigmoid(ih + ix  + i_s + self.bi)

        Th = tf.matmul(h_prev, self.W_Th)
        Tx = tf.matmul(input, self.W_Tx)
        Ts = tf.matmul(s, self.W_Ts)

        T = tf.sigmoid(Th + Tx  + Ts + self.bT)

        Eh = tf.matmul(h_prev, self.W_Eh)
        Ex = tf.matmul(input, self.W_Ex)
        Es = tf.matmul(s, self.W_Es)

        E = tf.tanh(Eh + Ex  + Es + self.bE)

        new_c = f * c_prev + i * E + T * s

        oh = tf.matmul(h_prev, self.W_oh)
        ox = tf.matmul(input, self.W_ox)
        os = tf.matmul(s, self.W_os)

        o = tf.sigmoid(oh + ox  + os + self.bo)

        _h = o * tf.math.tanh(new_c)

        e = self.attn(_h, new_c, mem)
        new_h = self.IM(input, _h, e)
        return new_h, new_c


In [8]:
@tf.keras.saving.register_keras_serializable()
class TH_LSTM(tf.keras.Model):
    def __init__(self, hidden_dim, output_dim, d,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.d = d

    def build(self, input_shape):
        self.hc_init = self.add_weight(name='h_init', shape = (1, self.hidden_dim), initializer='zeros')
        self.cell = cell(self.hidden_dim, self.d)
        self.output_dense = self.add_weight(name='op', shape = (self.hidden_dim, self.output_dim), initializer="random_uniform", trainable=True)
        self._init = self.add_weight(name='init', shape = (input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        return super().build(input_shape)
    
    def call(self, inputs):
        str_mem = tf.ones_like(inputs)
        str_mem = tf.matmul(str_mem, self._init)[:,:-1]
        
        h = c = self.hc_init
        for i in range(inputs.shape[1]):
            h, c = self.cell(inputs[:,i,..., 1:], h, c, inputs[:, i,... ,0:1], str_mem)
            str_mem = tf.concat((str_mem, h[:, tf.newaxis, ...]), axis = 1)[:,:self.d,...]
        output = tf.matmul(h, self.output_dense)
        return output

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [32]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, lambda1, lambda2):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.relu = tf.keras.layers.ReLU()
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.channel = 0

    def build(self, input_shape):
        if len(input_shape) == 3:
            self.channel = 1
        else:
            self.channel = input_shape[-1]
        self.W_t = self.add_weight(name='Wt', shape=(self.channel, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_s = self.add_weight(name='Ws', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        #self._ati_dense = self.add_weight(name='ati', shape=(input_shape[-2], self.hidden_dim), initializer="random_uniform", trainable=True)
        return super().build(input_shape)

    def call(self, inputs):
        if tf.rank(inputs) == 3:
            inputs = inputs[..., tf.newaxis]
        gt = tf.matmul(inputs, self.W_t)
        gt = self.relu(gt)
        a1 = tf.math.exp(gt * (1 - self.lambda1))
        a1 = a1 / tf.reduce_sum(a1, axis = 1)[:, tf.newaxis]
        rept = tf.tile(tf.reduce_sum(a1 * inputs, axis=1, keepdims=True), [1, inputs.shape[1], 1, 1])

        gs = tf.matmul(rept, self.W_s)
        gs = self.relu(gs)
        a2 = tf.math.exp(gs * (1 - self.lambda2))
        a2 = a2 / tf.reduce_sum(a2, axis = 2)[:, :, tf.newaxis]
        H = a2 * inputs
        return H

In [44]:
class STA_NN(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, lambda1, lambda2, num_c):
        super(STA_NN, self).__init__()
        self.attn = Attention(hidden_dim, lambda1, lambda2)
        self.conv2d = tf.keras.Sequential(
            [
            tf.keras.layers.Conv2D(filters = hidden_dim, kernel_size=3, padding='same')
            for i in range(num_c)
            ])
        self.top = tf.keras.Sequential(
            [
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(256, activation='relu'),
                tf.keras.layers.Dense(32, activation='relu'),
                tf.keras.layers.Dense(2, activation='softmax')
            ]
        )

    def call(self, inputs):
        output = self.attn(inputs)

        output = self.conv2d(output)

        output = self.top(output)

        return output

In [45]:
STA_NN(512, 0.8, 0.8, 10)(point)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.48859218, 0.5114078 ],
       [0.49352467, 0.5064753 ]], dtype=float32)>

In [35]:
t.fit(x=point, y=point[:,0,:], epochs=1)

NameError: name 't' is not defined

In [20]:
model.cell.W_Es

<tf.Variable 'cell_1/WEs:0' shape=(512, 512) dtype=float32, numpy=
array([[-0.04626746,  0.00359627, -0.01755141, ..., -0.02747763,
        -0.03189887, -0.03276787],
       [-0.01951   , -0.01298589, -0.01607934, ..., -0.01270718,
         0.05056564, -0.01247779],
       [-0.03104064, -0.00652095,  0.01753465, ..., -0.02565206,
         0.01810547, -0.03895471],
       ...,
       [ 0.03841003,  0.00052832, -0.04342259, ...,  0.03012684,
         0.03617146, -0.0062141 ],
       [-0.02431568,  0.0073331 ,  0.04505984, ...,  0.00349364,
        -0.00231323,  0.02137714],
       [ 0.04862043, -0.04206186,  0.02347884, ...,  0.04003372,
         0.03943432,  0.00278409]], dtype=float32)>

In [15]:
model = tf.keras.models.load_model('model.keras')


In [14]:
model.predict(point[0:1])

1/1 [==============================] - 0s 217ms/step


array([[ 0.31168705,  0.12628564, -0.39654619, -0.05241583,  0.30171004,
        -0.40108848, -0.1860197 ,  0.21693905, -0.24783114, -0.29487348,
         0.22307952,  0.37625092,  0.2241749 ,  0.3506708 ,  0.45104262,
        -0.56199753,  0.36240155, -0.09314545, -0.2025147 , -0.01523836,
         0.40751982,  0.35551974,  0.26489767,  0.05543795, -0.1606291 ,
        -0.20981666,  0.1846074 ,  0.4004755 , -0.11343882,  0.18619972,
        -0.22497378,  0.46918055, -0.28560185,  0.54379714,  0.27628478,
         0.27203625, -0.3975439 , -0.18416168, -0.16144982,  0.05874888,
        -0.28259882,  0.32966453, -0.34079683, -0.3700378 , -0.04298227,
        -0.02882063,  0.1003615 ,  0.4482764 , -0.03793479,  0.26154327,
        -0.03383964,  0.25987655,  0.14384776,  0.2574718 , -0.00138964,
         0.10461923, -0.09818178,  0.17011946, -0.49011055,  0.39715326,
         0.49603778, -0.23160155, -0.40634608, -0.26753786, -0.02015402,
        -0.04023924,  0.28586146, -0.28651053, -0.3

In [150]:
# Ma trận đầu vào
input_matrix = tf.random.uniform(shape=(10, 50, 512))

# Chuyển đổi ma trận bằng lớp Dense
output_matrix = dense_layer(input_matrix)

# Kích thước của ma trận đầu ra
print(output_matrix.shape)


(10, 50, 512)


In [1]:
import tensorflow as tf

In [19]:
class A_LSTM(tf.keras.Model):
    def __init__(self,num_classes,units, merge_units, dropout, head, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.head = head
        self.LSTM_blocks = [
            tf.keras.Sequential(
                [
                    tf.keras.layers.LSTM(units=units, return_sequences=True),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.LSTM(units=units*2),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(merge_units, activation='relu')
                ]
            )
        for i in range(head)]
        self.forward = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(merge_units, activation='relu'),
                tf.keras.layers.Dense(num_classes, activation='softmax')
            ]
        )

    def call(self, inputs):
        _dense_output = []
        for i in range(self.head):
            otp = self.LSTM_blocks[i](inputs[i])
            _dense_output.append(otp)
        _dense_output = tf.concat(_dense_output, axis=-1)
        output = self.forward(_dense_output)
        return output

In [13]:
import numpy as np 
point = np.zeros((2, 50, 30))

In [20]:
A_LSTM(2, 128, 64, 0.5, 3)([point, point, point])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.5, 0.5],
       [0.5, 0.5]], dtype=float32)>

In [10]:
attn._combine_equation

'acbe,aecd->abcd'

In [11]:
attn._dot_product_equation

'aecd,abcd->acbe'

In [20]:
query = attn._query_dense(h)

In [39]:
query

<tf.Tensor: shape=(2, 50, 8, 512), dtype=float32, numpy=
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 

In [21]:
key = attn._key_dense(h)

In [22]:
value = attn._value_dense(c)

In [25]:
t = tf.einsum(attn._dot_product_equation, key, query)

In [27]:
t

<tf.Tensor: shape=(2, 8, 50, 50), dtype=float32, numpy=
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0

In [26]:
tf.einsum(attn._combine_equation, t, value)

<tf.Tensor: shape=(2, 50, 8, 512), dtype=float32, numpy=
array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 

In [ ]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, d):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.d = d

    def build(self, input_shape):
        self.W_as = self.add_weight(name='Was', shape=(self.hidden_dim * 2, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_ah = self.add_weight(name='Wah', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.ba = self.add_weight(name='ba', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.v = self.add_weight(name='v', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)
        #self._ati_dense = self.add_weight(name='ati', shape=(input_shape[-2], self.hidden_dim), initializer="random_uniform", trainable=True)
        return super().build(input_shape)

    def call(self, o, ct, hi):
        ht = o * tf.tanh(ct)
        q = tf.concat((ht, ct), axis = -1)
        as = tf.matmul(q, self.W_as)
        ah = tf.matmul(hi, self.W_ah)

        oti = tf.math.tanh(as[:, tf.newaxis, ...] + ah + self.ba)
        dims=tf.range(len(tf.shape(oti)))
        perm = tf.concat((dims[:-2], dims[-1:], dims[-2:-1]), axis = 0)
        oti = tf.transpose(oti, perm=perm)

        a = tf.matmul(self.v, oti)[:,0]
        ati = tf.exp(a)
        ati = ati / tf.reduce_sum(ati, axis=1)[:, tf.newaxis, ...]

        #ati = tf.matmul(ati, self._ati_dense)
        e = tf.reduce_sum(ati[..., tf.newaxis] * hi, axis=1)
        return e

In [ ]:
class cell(tf.keras.layers.Layer):
    def __init__(self, hidden_dim, d, **kwargs):
        super().__init__( **kwargs)
        self.hidden_dim = hidden_dim
        self.attn = Attention(hidden_dim, d)
        self.IM = InteractionModule(hidden_dim)
        self.d = d

    def build(self, input_shape):
        self.W_fh = self.add_weight(name='Wfh', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_fx = self.add_weight(name='Wfx', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_ft = self.add_weight(name='Wft', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bf = self.add_weight(name='bf', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_ih = self.add_weight(name='Wih', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_ix = self.add_weight(name='Wix', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bi = self.add_weight(name='bi', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_oh = self.add_weight(name='Woh', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_ox = self.add_weight(name='Wox', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bo = self.add_weight(name='bo', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)

        self.W_ch = self.add_weight(name='Wch', shape=(self.hidden_dim, self.hidden_dim), initializer="random_uniform", trainable=True)
        self.W_cx = self.add_weight(name='Wcx', shape=(input_shape[-1], self.hidden_dim), initializer="random_uniform", trainable=True)
        self.bc = self.add_weight(name='bo', shape=(1, self.hidden_dim), initializer="random_uniform", trainable=True)
        return super().build(input_shape)
    
    def call(self, input, h_prev, c_prev, delta_t, mem):
        fh = tf.matmul(h_prev, self.W_fh)
        fx = tf.matmul(input, self.W_fx)
        ft = tf.matmul(delta_t, self.W_ft)

        f = tf.sigmoid(fh + fx  + ft + self.bf)

        ih = tf.matmul(h_prev, self.W_ih)
        ix = tf.matmul(input, self.W_ix)

        i = tf.sigmoid(ih + ix + self.bi)

        oh = tf.matmul(h_prev, self.W_oh)
        ox = tf.matmul(input, self.W_ox)

        o = tf.sigmoid(oh + ox + self.bo)

        _ch = tf.matmul(self.W_ch, h_prev)
        _cx = tf.matmul(self.W_cx, input)
        _c = tf.tanh(_ch + _cx + self.bc)

        new_c = f * c_prev + i * _c

        e = self.attn(o, new_c, mem)
        new_h = self.IM(input, _h, e)
        return new_h, new_c
